# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qaqortoq,60.7167,-46.0333,-16.81,66,97,4.59,GL,1738694989
1,1,punta arenas,-53.1500,-70.9167,13.06,54,75,12.35,CL,1738694901
2,2,avarua,-21.2078,-159.7750,29.03,79,88,6.69,CK,1738694991
3,3,fort bragg,35.1390,-79.0060,24.91,37,0,5.66,US,1738694992
4,4,saipan,15.1355,145.7010,27.39,74,0,3.60,MP,1738694994


In [3]:
import hvplot.pandas
import geoviews as gv
import holoviews as hv
hv.extension('bokeh') 

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    size='Humidity',  # The size of the points is based on humidity
    color='Humidity',  # Color represents humidity level
    cmap='coolwarm',  # Color gradient
    tiles='CartoLight',  # Map background
    frame_width=800, 
    frame_height=500, 
    hover_cols=['City', 'Humidity']
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[(city_data_df['Humidity'] < 70) & (city_data_df['Max Temp'] > 20)]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()
# Display sample data
filtered_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,fort bragg,35.1390,-79.0060,24.91,37,0,5.66,US,1738694992
13,13,santa maria,-29.6842,-53.8069,37.61,32,40,2.57,BR,1738694708
16,16,massawa,15.6097,39.4500,26.95,66,0,1.17,ER,1738695009
18,18,kingston,17.9970,-76.7936,30.09,62,40,2.06,JM,1738694874
30,30,montes claros,-16.7350,-43.8617,31.88,35,20,3.60,BR,1738694467


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,qaqortoq,GL,60.7167,-46.0333,66,
1,punta arenas,CL,-53.1500,-70.9167,54,
2,avarua,CK,-21.2078,-159.7750,79,
3,fort bragg,US,35.1390,-79.0060,37,
4,saipan,MP,15.1355,145.7010,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel

radius = 10000
params = {
    "categories": "accommodation.hotel",  # Search for hotels
    "apiKey":geoapify_key, # Your API key
    "limit": 1,  # Get only the first result
    "filter": "",  # Placeholder, updated per city
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
qaqortoq - nearest hotel: Hotel Qaqortoq
punta arenas - nearest hotel: Hotel Lacolet
avarua - nearest hotel: Paradise Inn
fort bragg - nearest hotel: Airborne Inn Lodging
saipan - nearest hotel: Chalan Kanoa Beach Hotel
lebu - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
turpan - nearest hotel: Hua Sheng Hotel
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
papatowai - nearest hotel: No hotel found
bel ombre - nearest hotel: Villa La Cachette
okha - nearest hotel: Сфера
port-aux-francais - nearest hotel: Keravel
santa maria - nearest hotel: Hotel Morotin
thompson - nearest hotel: Thompson Inn
susuman - nearest hotel: No hotel found
massawa - nearest hotel: Seghen International Hotel فندق دولي سيغن
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
kingston - nearest hotel: Aphrodite Hotel
invercargill - nearest hotel: The Grand
arkalyk - nearest hotel: Гостиница "Арсенал"
waitangi - nearest hotel: Hotel Chathams
georgetow

,City,Country,Lat,Lng,Humidity,Hotel Name
0,qaqortoq,GL,60.7167,-46.0333,66,Hotel Qaqortoq
1,punta arenas,CL,-53.1500,-70.9167,54,Hotel Lacolet
2,avarua,CK,-21.2078,-159.7750,79,Paradise Inn
3,fort bragg,US,35.1390,-79.0060,37,Airborne Inn Lodging
4,saipan,MP,15.1355,145.7010,74,Chalan Kanoa Beach Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
city_map = hotel_df.hvplot.points(
    x='Lng',  # Longitude for x-axis
    y='Lat',  # Latitude for y-axis
    size='Humidity',  # Size of the points based on humidity
    color='Humidity',  # Color of the points based on humidity
    cmap='coolwarm',  # Color gradient from cool to warm
    tiles='CartoDark',  # Dark background for better contrast
    frame_width=800, 
    frame_height=500, 
    hover_cols=['City', 'Hotel Name', 'Hotel Address'],  # Show city, hotel name, and address
    title="Cities and Nearby Hotels"
)

# Display the map
city_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (Humidity,City,Hotel Name)